# Thống kê mô tả dữ liệu COVID-19 (Chi tiết)

Notebook này thực hiện thống kê mô tả chuyên sâu cho hai bộ dữ liệu:

1. **01_clean_daily_timeseries.csv.gz** – dữ liệu theo ngày  
2. **02_country_population_summary.csv.gz** – dữ liệu tổng hợp theo quốc gia  

Nội dung phân tích gồm:
- Thống kê mô tả: mean, median, mode, std, min, max  
- Range, IQR, Skewness, Kurtosis  
- Coefficient of Variation (CV)  
- Phân tích phân phối  
- Kiểm tra outliers theo IQR  
- Nhận xét chuyên sâu

Các chỉ số giúp hiểu rõ phân phối dữ liệu, mức độ biến động và độ lệch, từ đó hỗ trợ phân tích dịch tễ tốt hơn.


# 1. Import thư viện

In [14]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

# 2. Load dữ liệu CSV


In [15]:
df_ts = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/01_clean_daily_timeseries.csv.gz")
df_pop = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/02_country_population_summary.csv.gz")

df_ts.head(), df_pop.head()

(  Date_reported Country_code      Country WHO_region  New_cases  New_deaths  \
 0    2020-01-04           AF  Afghanistan        EMR        1.0         1.0   
 1    2020-01-05           AF  Afghanistan        EMR        1.0         1.0   
 2    2020-01-06           AF  Afghanistan        EMR        1.0         1.0   
 3    2020-01-07           AF  Afghanistan        EMR        1.0         1.0   
 4    2020-01-08           AF  Afghanistan        EMR        1.0         1.0   
 
    Cumulative_cases  Cumulative_deaths  New_cases_MA7  New_deaths_MA7  \
 0                 0                  0            1.0             1.0   
 1                 0                  0            1.0             1.0   
 2                 0                  0            1.0             1.0   
 3                 0                  0            1.0             1.0   
 4                 0                  0            1.0             1.0   
 
    Growth_Rate  
 0          0.0  
 1          0.0  
 2          0.0  


# 3. Hàm tính thống kê mô tả nâng cao

Bao gồm:
- Mean, Median, Mode
- Std, Min, Max
- Range
- IQR
- Skewness (độ lệch)
- Kurtosis (độ nhọn)
- CV (Coefficient of Variation)

In [16]:
def extended_descriptive_stats(df):
    result = pd.DataFrame(index=df.columns)

    result["count"] = df.count()
    result["mean"] = df.mean()
    result["median"] = df.median()
    result["mode"] = df.mode().iloc[0]
    result["std"] = df.std()
    result["min"] = df.min()
    result["25%"] = df.quantile(0.25)
    result["75%"] = df.quantile(0.75)
    result["max"] = df.max()

    result["range"] = result["max"] - result["min"]
    result["IQR"] = result["75%"] - result["25%"]
    result["skewness"] = df.apply(skew)
    result["kurtosis"] = df.apply(kurtosis)
    result["CV"] = result["std"] / result["mean"]

    return result

# 4. Thống kê mô tả chi tiết – Dataset 1 (Daily Timeseries)

Chỉ áp dụng cho các cột dạng số.

In [17]:
num_ts = df_ts.select_dtypes(include='number')
stats_ts = extended_descriptive_stats(num_ts)
stats_ts

,count,mean,median,mode,std,min,25%,75%,max,range,IQR,skewness,kurtosis,CV
New_cases,502800,3.568052e+03,27.0,0.0,4.315891e+04,0.0,2.000000,299.000000,6.966046e+06,6.966046e+06,297.000000,61.765092,6600.359147,12.095928
New_deaths,502800,2.870821e+01,1.0,1.0,1.951661e+02,0.0,0.000000,4.000000,4.404700e+04,4.404700e+04,4.000000,36.165608,5429.212730,6.798267
Cumulative_cases,502800,2.076666e+06,62488.0,0.0,8.610268e+06,0.0,6771.000000,664972.750000,1.034368e+08,1.034368e+08,658201.750000,8.104672,79.705834,4.146199
Cumulative_deaths,502800,2.188705e+04,706.0,0.0,8.743394e+04,0.0,41.000000,8011.250000,1.228289e+06,1.228289e+06,7970.250000,8.353488,87.118787,3.994779
New_cases_MA7,502800,3.567081e+03,29.0,0.0,4.232774e+04,0.0,2.571429,322.969388,5.882129e+06,5.882129e+06,320.397959,57.503508,5545.946727,11.866210
New_deaths_MA7,502800,2.871128e+01,1.0,1.0,1.826060e+02,0.0,0.142857,4.000000,6.829714e+03,6.829714e+03,3.857143,15.134629,314.308360,6.360081
Growth_Rate,502800,1.259505e+01,0.0,0.0,5.083364e+02,-100.0,-2.517259,1.685536,2.137000e+05,2.138000e+05,4.202795,258.929834,88126.080155,40.360017


## Nhận xét chuyên sâu – Daily Timeseries

### 1. Skewness (độ lệch)
- Hầu hết biến có **skewness dương rất lớn** → phân phối lệch phải mạnh.
- Nguyên nhân: số ít quốc gia có số ca rất lớn.

### 2. Kurtosis (độ nhọn)
- Kurtosis cao (>10 ở nhiều biến) → phân phối có "đuôi dày", nhiều outliers.

### 3. Mode = 0 ở nhiều biến
→ Phần lớn ngày không có ca mới hoặc tử vong.

### 4. Coefficient of Variation (CV)
- CV rất lớn → biến động mạnh, không ổn định theo thời gian.

### 5. Range & IQR
- Range cực kỳ lớn → dữ liệu trải dài do có quốc gia bùng dịch mạnh.
- IQR nhỏ → phần lớn quốc gia có ca số thấp.


# 5. Kiểm tra Outliers – Dataset 1
Dùng phương pháp IQR:
Outlier nếu:  
> value < Q1 - 1.5×IQR  
hoặc  
> value > Q3 + 1.5×IQR


In [18]:
def count_outliers(df):
    outlier_counts = {}
    for col in df.columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        outlier_counts[col] = ((df[col] < lower) | (df[col] > upper)).sum()
    return pd.DataFrame.from_dict(outlier_counts, orient='index', columns=["outlier_count"])

outliers_ts = count_outliers(num_ts)
outliers_ts

,outlier_count
New_cases,89387
New_deaths,83207
Cumulative_cases,75528
Cumulative_deaths,76971
New_cases_MA7,88684
New_deaths_MA7,88159
Growth_Rate,165494


# 6. Thống kê mô tả chi tiết – Dataset 2 (Country Population Summary)

In [19]:
num_pop = df_pop.select_dtypes(include='number')
stats_pop = extended_descriptive_stats(num_pop)
stats_pop

,count,mean,median,mode,std,min,25%,75%,max,range,IQR,skewness,kurtosis,CV
Total_Cases,238,3.272021e+06,1.739410e+05,0.0,1.134708e+07,0.0,22527.000000,1.254029e+06,1.034368e+08,1.034368e+08,1.231502e+06,6.337490,47.171557,3.467911
Total_Deaths,238,2.984295e+04,1.471000e+03,0.0,1.080736e+05,0.0,161.500000,1.260250e+04,1.228289e+06,1.228289e+06,1.244100e+04,7.596440,70.528150,3.621412
Population,238,3.375597e+07,5.265350e+06,6000.0,1.361235e+08,50.0,332806.750000,2.180387e+07,1.438070e+09,1.438070e+09,2.147106e+07,9.134846,89.575735,4.032576
Cases_per_1M,238,2.017752e+05,1.350278e+05,0.0,2.020834e+05,0.0,16449.561564,3.437774e+05,7.638104e+05,7.638104e+05,3.273278e+05,0.805108,-0.528836,1.001527
Deaths_per_1M,238,1.245832e+03,8.258387e+02,0.0,1.349493e+03,0.0,123.966919,1.971679e+03,6.531422e+03,6.531422e+03,1.847712e+03,1.345334,1.522562,1.083206
Fatality_Rate,236,1.210699e+00,8.253091e-01,0.0,1.556502e+00,0.0,0.371598,1.617082e+00,1.807451e+01,1.807451e+01,1.245484e+00,NaN,NaN,1.285623


## Nhận xét chuyên sâu – Population Summary

### 1. Skewness và Kurtosis rất cao  
→ Phân phối lệch phải mạnh (đặc biệt Total Cases, Total Deaths).

### 2. CV lớn → biến động mạnh giữa các quốc gia.

### 3. Range và Max lớn bất thường  
→ xuất hiện cực trị từ các quốc gia đông dân hoặc ảnh hưởng COVID lớn.

### 4. Fatality Rate có kurtosis cao  
→ Nhiều giá trị bất thường ở quốc gia có số ca thấp.

### 5. Cases per 1M và Deaths per 1M  
→ Phân phối hợp lý hơn so với số ca tuyệt đối.


# 7. Outliers dataset 2

In [20]:
outliers_pop = count_outliers(num_pop)
outliers_pop

,outlier_count
Total_Cases,39
Total_Deaths,35
Population,27
Cases_per_1M,0
Deaths_per_1M,7
Fatality_Rate,9


# 8. KẾT LUẬN CHUNG

- Cả hai dataset đều có phân phối **không chuẩn**, **lệch phải mạnh**, nhiều outliers.
- Mean không phản ánh đúng dữ liệu → nên dùng median hoặc trimmed mean.
- Biến được chuẩn hóa theo dân số (per 1M) phù hợp để so sánh quốc gia.
- Growth Rate và Fatality Rate dễ nhiễu → cần lọc outliers trước khi phân tích.
- CV lớn và IQR nhỏ → dữ liệu phân tán mạnh nhưng phần lớn tập trung ở vùng giá trị thấp.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load dữ liệu (Giả sử bạn đã có file csv)
# df = pd.read_csv('WHO-COVID-19-global-data.csv')

# --- THỐNG KÊ CƠ BẢN ---
print("=== THÔNG TIN BỘ DỮ LIỆU ===")
print(f"Tổng số dòng: {len(df)}")
print(f"Số lượng quốc gia: {df['Country'].nunique()}")
print(f"Ngày bắt đầu: {df['Date_reported'].min()}")
print(f"Ngày kết thúc: {df['Date_reported'].max()}")

# Kiểm tra giá trị âm
neg_cases = df[df['New_cases'] < 0].shape[0]
neg_deaths = df[df['New_deaths'] < 0].shape[0]
print(f"\nSố dòng có ca nhiễm âm: {neg_cases}")
print(f"Số dòng có tử vong âm: {neg_deaths}")

# --- TIỀN XỬ LÝ SƠ BỘ ---
# 1. Chuyển giá trị âm thành NaN (để nội suy) hoặc 0
df['New_cases'] = df['New_cases'].apply(lambda x: x if x >= 0 else None)
df['New_deaths'] = df['New_deaths'].apply(lambda x: x if x >= 0 else None)

# 2. Xử lý Missing Values (Nội suy theo từng quốc gia)
# Lưu ý: Phải group by Country trước khi interpolate để không bị lẫn số liệu giữa các nước
df['New_cases'] = df.groupby('Country')['New_cases'].transform(lambda x: x.interpolate(method='linear').fillna(0))
df['New_deaths'] = df.groupby('Country')['New_deaths'].transform(lambda x: x.interpolate(method='linear').fillna(0))

print("\nĐã xử lý xong giá trị âm và missing values.")

# --- VẼ BIỂU ĐỒ PHÂN PHỐI (HISTOGRAM) ---
# Mục đích: Chứng minh dữ liệu bị lệch (Skewed), cần phân cụm
# Lấy tổng ca nhiễm của mỗi nước để vẽ
country_stats = df.groupby('Country')['Cumulative_cases'].max().reset_index()

plt.figure(figsize=(10, 6))
sns.histplot(country_stats['Cumulative_cases'], bins=30, kde=True, color='skyblue')
plt.title('Phân phối Tổng số ca nhiễm của các quốc gia')
plt.xlabel('Tổng ca nhiễm (Cumulative Cases)')
plt.ylabel('Số lượng quốc gia')
plt.grid(True, linestyle='--', alpha=0.5)

# Dùng log scale để nhìn rõ hơn (vì chênh lệch quá lớn)
plt.xscale('log') 
plt.xlabel('Tổng ca nhiễm (Log Scale)')
plt.show()